In [336]:
from nltk.tokenize import word_tokenize
import nltk
import re
import numpy
import fasttext
import pandas

In [337]:
def tokenization(query):
    tokens=query.upper()
    #'``': 'DQUT', '\'\'': 'DQUT',
    Dict = { '!=': 'NEQ', '<>': 'NEQ', '&&': 'AND', '||': 'OR', '/*': ' CMTST ',
            '*/': 'CMTEND',
            '~': 'TLDE', '!': 'EXCLM', '@': 'ATR', '#': 'HASH', '$': 'DLLR', '%': 'PRCNT', '^': 'XOR', '&': 'BITAND',
            '|': 'BITOR', '*': 'STAR', '--': 'CMNT', '-': 'MINUS', '+': 'PLUS', '=': 'EQ', '/': 'SLSH', '?': 'QSTN',
            '.': 'DOT',
            ',': 'CMMA', '>': 'GT', '<': 'LT', '\'': 'SQUT', '"': 'DQUT', ';': 'SMCLN', ':': 'CLN', '\\': 'BSLSH',
            ']': 'RSQBR', '[': 'LSQBR', '}': 'RCBR', '{': 'LCBR', ')': 'RPRN', '(': 'LPRN'}
    i=0
    while i < len(tokens):
        if tokens[i] in Dict:
            if i + 1 < len(tokens) and (tokens[i] + tokens[i + 1]) in Dict:
                tokens=tokens.replace((tokens[i] + tokens[i + 1])," "+Dict[(tokens[i] + tokens[i + 1])]+" ")
            else:
                tokens=tokens.replace(tokens[i], " "+Dict[tokens[i]]+" ")
           
        i = i + 1
    #print(tokens)    
    tokens=word_tokenize(tokens)
    while i < len(tokens):
        if re.search("^((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$", tokens[i]):
            tokens[i] = "IPADDR"
        elif re.search("^([0-9]+)$", tokens[i]):
            tokens[i] = "INT"
        elif re.search("^([A-Fa-f0-9]{2,})$", tokens[i]):
            tokens[i] = "HEX"
        elif re.search("^\d*[.,]?\d*$", tokens[i]):
            tokens[i] = "DEC"
        elif re.search("^([A-Za-z])$", tokens[i]):
            tokens[i] = "CHR"
        else:
            if i-1 > 0 and tokens[i-1] == "FROM":
                tokens[i]= "TABLE"     
    return " ".join(tokens)

In [338]:
queries = open("/home/mariam/Desktop/dataset/queries.txt", "rt")
tokenized = open("/home/mariam/Desktop/dataset/tokenized.txt", "w")

In [339]:
counter=0
while True:
    counter=counter+1
    line=queries.readline()
    if not line:
        break
    line.strip()
    line=line.split(" ",1)
    query=tokenization(line[1])
    tokenized.writelines([line[0]+" "+query,"\n"])
queries.close()
tokenized.close() 

In [340]:
training=open("/home/mariam/Desktop/dataset/train.txt", "w") #450 malicious #407 non-malicious
valid=open("/home/mariam/Desktop/dataset/valid.txt", "w") # 150 malicious #100 non-malicios
tokenized = open("/home/mariam/Desktop/dataset/tokenized.txt", "r")


In [341]:
counter=0
while True:
    counter=counter+1
    line=tokenized.readline()
    if not line:
        break
    line.strip()
    #label_query='__label__nonmalicious  '+ line
    if counter<=407:
        training.writelines([line,"\n"])
    elif counter>407 and counter<=507:
        valid.writelines([line,"\n"])
    #label_malicious='__label__malicious  '+ line
    if counter>507 and counter<=957:
        training.writelines([line,"\n"])
    elif counter>957 and counter<=1107:
        valid.writelines([line,"\n"])

        
        
valid.close()    
tokenized.close()
training.close()

In [342]:
model = fasttext.train_supervised('/home/mariam/Desktop/dataset/train.txt')

In [343]:
#print(model.test('/home/mariam/Desktop/dataset/valid.txt'))
total, precision, recall = model.test('/home/mariam/Desktop/dataset/valid.txt')
Fmeasure= 2 * ((precision*recall)/(precision + recall))
print(str(Fmeasure * 100) + "%" )

97.2%


In [303]:
model.predict("INT '' RPRN RPRN RPRN OR EXTRACTVALUE LPRN INT CMMA CONCAT LPRN 0X5C,0X7171706A71 CMMA LPRN SELECT LPRN ELT LPRN INT EQ DEC RPRN RPRN RPRN ,0X717A767A71 RPRN RPRN AND LPRN LPRN LPRN `` YXPO '' LIKE `` YXPO")

(('__label__malicious',), array([0.99063867]))

In [304]:
model.predict("SELECT COUNT LPRN STAR RPRN AS DISTINCTCOUNTRIES FROM LPRN ;")

(('__label__nonmalicious',), array([0.8489356]))

In [305]:
model.predict(tokenization('1%"  )  )   )  union all select null,null,null,null,null,null,null,null--'))

(('__label__malicious',), array([0.98988789]))

In [306]:
model.predict(tokenization("SELECT * FROM () Customers WHERE CustomerName LIKE '_r%';"))

(('__label__nonmalicious',), array([0.99196416]))

In [307]:
model.predict(tokenization(" 1 )  as bidc where 5268 = 5268 and  ( select * from  ( select ( sleep ( 5  )  )   ) fzno ) --"))

(('__label__malicious',), array([0.97067046]))

In [308]:
model.predict(tokenization("SELECT * FROM rnoo WHERE rnoo.haya != rnoo.leen and Mariam = 5;"))

(('__label__nonmalicious',), array([0.99037927]))

In [309]:
model.predict(tokenization("hello ghkdh jgfk ldhfnvfk"))

(('__label__nonmalicious',), array([0.73469847]))

In [310]:
model.predict(tokenization('1%"  )  )   or 7552 =  ( select count ( * )  from rdb$fields as t1,rdb$types as t2,rdb$collations as t3,rdb$functions as t4 )  and   (  (  "%" = "'))

(('__label__malicious',), array([0.95017999]))

In [313]:
model.predict(tokenization("UPDATE users SET email = lower(email);"))

(('__label__nonmalicious',), array([0.59816259]))

In [314]:
model.predict(tokenization("DROP TABLE IF EXISTS customer CASCADE;"))

(('__label__nonmalicious',), array([0.98939621]))

In [316]:
model.predict(tokenization("INSERT INTO product (name, price) SELECT random()::text, (random() * 1000)::int FROM generate_series(0, 10000);"))

(('__label__malicious',), array([0.56772649]))

In [317]:
model.predict(tokenization("select * from users where id = 1 or  ( \. )  = 1 union select 1,@@VERSION -- 1"))

(('__label__malicious',), array([0.97918332]))

In [318]:
model.predict(tokenization("declare @q nvarchar  ( 200 )  0x730065006c00650063007400200040004000760065007200730069006f006e00 exec ( @q )"))

(('__label__malicious',), array([0.98334169]))

In [319]:
model.predict(tokenization("or a  =  a"))

(('__label__malicious',), array([0.99375033]))

In [320]:
model.predict(tokenization('admin" #'))

(('__label__malicious',), array([0.9248324]))

In [321]:
model.predict(tokenization("1  )  )   as dgso where 4654 = 4654 and 2853 = cast  (  (  chr ( 113 ) ||chr ( 113 ) ||chr ( 112 ) ||chr ( 106 ) ||chr ( 113  )  )  || ( select  ( case when  ( 2853 = 2853 )  then 1 else 0 end  )  )  ::text|| ( chr ( 113 ) ||chr ( 122 ) ||chr ( 118 ) ||chr ( 122 ) ||chr ( 113  )  )   as numeric ) --"))

(('__label__malicious',), array([0.99449897]))

In [322]:
model.predict(tokenization("INSERT INTO customer (name) SELECT random()::text FROM generate_series(0, 100000);"))

(('__label__nonmalicious',), array([0.65064132]))

In [327]:
model.predict(tokenization("insert into users ( login, password, level )  values (  char ( 0x70 )  + char ( 0x65 )  + char ( 0x74 )  + char ( 0x65 )  + char ( 0x72 )  + char ( 0x70 )  + char ( 0x65 )  + char ( 0x74 )  + char ( 0x65 )  + char ( 0x72 ) ,char ( 0x64 )"))

(('__label__malicious',), array([0.98365986]))

In [326]:
model.predict(tokenization("insert into mysql.user  ( user, host, password )  values  ( 'name', 'localhost', password ( 'pass123'  )  ) "))

(('__label__malicious',), array([0.80118978]))

In [292]:
model.get_word_vector("SELECT")

array([ 0.11358951,  0.10480653, -0.05767942, -0.12872459,  0.28193882,
        0.1448833 , -0.15858439, -0.22040069,  0.29239404,  0.29715583,
       -0.23745717,  0.08715382, -0.14158145,  0.13052242, -0.06275795,
        0.04183234, -0.06545533, -0.16901596, -0.06749158,  0.0792063 ,
        0.22366424,  0.16209778,  0.0024659 , -0.11143451, -0.09987728,
       -0.05727419,  0.30393136, -0.26309544, -0.16630766,  0.14585237,
        0.12086555,  0.00768222, -0.18256299, -0.10169935, -0.29963878,
       -0.0056648 , -0.1510986 ,  0.00072369,  0.0390562 ,  0.10351729,
       -0.15258197,  0.25789902,  0.14535214,  0.13704202,  0.2347164 ,
       -0.17208758,  0.23449722,  0.06946068, -0.02486586,  0.18184304,
       -0.30472103,  0.12131523, -0.04642637, -0.02723935, -0.11215311,
        0.02186445, -0.13916573,  0.10977095,  0.1335517 ,  0.10741097,
       -0.03336021,  0.2084495 ,  0.22489786,  0.24140877, -0.16731335,
        0.05025179, -0.20592092, -0.06136441, -0.11144574,  0.03